Fine-tuning of gemma-2b-it

In [1]:
# ALL THE NECESSARY IMPORTS

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm

from dataclasses import dataclass, field
from typing import Optional
from sklearn.model_selection import train_test_split


from datasets import load_dataset
from functools import partial
from peft import LoraConfig, TaskType, get_peft_model, get_peft_config

# Filepath to embeddings
fname = "/mnt/mimic/data/HAIM/mimic_extras/embeddings.csv"

Setting up the model

Different versions, with huggingface LoRA-class or custom Adapter-module.

In [3]:
# LoRA parameter efficient fine-tuning
# Parameters are freezed and small submodules with low-rank matrices ar inserted at the target layers.
# initialization of model
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
tokenizer.pad_token_id = tokenizer.eos_token_id
gemma = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", quantization_config=quantization_config,attn_implementation="sdpa")
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
    lora_alpha=16,
    lora_dropout=0.1
)

gemma = get_peft_model(gemma, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Model-structure and trainable parameters (this can be tuned by hyperparameters)
gemma.print_trainable_parameters()
gemma

trainable params: 9,805,824 || all params: 2,515,978,240 || trainable%: 0.3897420034920493


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [2]:
# Adapter NN for parameter efficient fine-tuning
# Adapters (bottleneck feed-forward networks) are added as modules to the layers of the model
# adapting attention projections and MLP projections while freezing original model parameters

class Adapter(nn.Module):
    def __init__(self, size = 6, model_dim = 2048):
        super().__init__()
        self.adapter_block = nn.Sequential(
            nn.Linear(model_dim, size),
            nn.ReLU(),
            nn.Linear(size, model_dim)
        )

    def forward(self, x):

        output = self.adapter_block(x)
        adapter_out = output + x

        return adapter_out


class Adaptered(nn.Module):
    def __init__(self, orig_layer):
        super().__init__()
        self.orig_layer = orig_layer
        self.adapter = Adapter()

    def forward(self, *x):
        orig_out = self.orig_layer(*x)
        output = (self.adapter.forward(orig_out[0].unsqueeze(0))[0],)

        return output



class model_with_adapter(nn.Module):

    def __init__(self):
        super().__init__()
        self.quantization_config = BitsAndBytesConfig(load_in_4bit=True)
        self.model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", quantization_config=self.quantization_config,attn_implementation="sdpa")
        # Freeze the original model parameters
        for params in self.model.parameters():
            params.requires_grad = False
        # Embed adapter layers into the transformer blocks 
        for i, gemma_layer in enumerate(self.model.model.layers):
            gemma_layer.self_attn.q_proj = Adaptered(gemma_layer.self_attn.q_proj)
            gemma_layer.self_attn.k_proj = Adaptered(gemma_layer.self_attn.k_proj)
            gemma_layer.self_attn.v_proj = Adaptered(gemma_layer.self_attn.v_proj)
            gemma_layer.self_attn.o_proj = Adaptered(gemma_layer.self_attn.o_proj)
    
            gemma_layer.mlp.gate_proj = Adaptered(gemma_layer.mlp.gate_proj)
            gemma_layer.mlp.up_proj = Adaptered(gemma_layer.mlp.up_proj)
            gemma_layer.mlp.down_proj = Adaptered(gemma_layer.mlp.down_proj)

    def get_model(self):

        return self.model



In [3]:
# Custom get_parameters function
def get_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_percentage = (trainable_params / total_params) * 100

    trainable_params_str = "{:,}".format(trainable_params)
    total_params_str = "{:,}".format(total_params)

    print(f"trainable params: {trainable_params_str} || all params: {total_params_str} || trainable%: {trainable_percentage}")

In [4]:
# Initialization of adapter-model.
# 
gemma = model_with_adapter().to('cuda')

# Model-structure and trainable parameters (this can be tuned by hyperparameters)
get_parameters(gemma)
gemma

# OBS A lot less params, not sure why.. (maybe cause of degeneration? or mistake, need to look into)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 3,355,380 || all params: 1,518,623,476 || trainable%: 0.2209487771674616


model_with_adapter(
  (model): GemmaForCausalLM(
    (model): GemmaModel(
      (embed_tokens): Embedding(256000, 2048, padding_idx=0)
      (layers): ModuleList(
        (0-17): 18 x GemmaDecoderLayer(
          (self_attn): GemmaSdpaAttention(
            (q_proj): Adaptered(
              (orig_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
              (adapter): Adapter(
                (adapter_block): Sequential(
                  (0): Linear(in_features=2048, out_features=6, bias=True)
                  (1): ReLU()
                  (2): Linear(in_features=6, out_features=2048, bias=True)
                )
              )
            )
            (k_proj): Adaptered(
              (orig_layer): Linear4bit(in_features=2048, out_features=256, bias=False)
              (adapter): Adapter(
                (adapter_block): Sequential(
                  (0): Linear(in_features=2048, out_features=6, bias=True)
                  (1): ReLU()
                  (2):

Projection module

In [5]:
embedding_size = 1024
projection_size = 6

class ProjectionNN(nn.Module):
    def __init__(self):
        super(ProjectionNN, self).__init__()

        # Architecture
        self.fc1 = nn.Linear(embedding_size, 128).cuda()
        self.relu = nn.ReLU().cuda()
        self.fc2 = nn.Linear(128, 2048 * projection_size).cuda()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = x.view(-1,6,2048)
        return x

Fetching and preprocessing of data

In [6]:
df = pd.read_csv(fname)
df_death_small48 = df[((df['img_length_of_stay'] < 48) & (df['death_status'] == 1))]
df_alive_big48 = df[((df['img_length_of_stay'] >= 48) & (df['death_status'] == 0))]
df_death_big48 = df[((df['img_length_of_stay'] >= 48) & (df['death_status'] == 1))]

df_death_small48['y'] = 1
df_alive_big48['y'] = 0
df_death_big48['y'] = 0
df = pd.concat([df_death_small48, df_alive_big48, df_death_big48], axis = 0)

/tmp/ipykernel_541228/4265014453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_death_small48['y'] = 1
/tmp/ipykernel_541228/4265014453.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alive_big48['y'] = 0
/tmp/ipykernel_541228/4265014453.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [7]:
vd_cols = df.filter(regex='^vd_')
y_col = df[['y']]
haim_col = df[['haim_id']]
df = pd.concat([haim_col, vd_cols, y_col], axis=1)

pkl_list = df['haim_id'].unique().tolist()

print(df.head())

     haim_id      vd_0      vd_1      vd_2      vd_3      vd_4      vd_5  \
256     6557  0.005299  0.082119  0.274407  0.017487  0.255308  0.003707   
259     6557  0.000000  0.079306  0.381579  0.015250  0.402685  0.011122   
267     6558  0.005299  0.082119  0.274407  0.017487  0.255308  0.003707   
270     6558  0.000000  0.079306  0.381579  0.015250  0.402685  0.011122   
319     6581  0.002288  0.078941  0.088397  0.017775  0.071482  0.006970   

         vd_6      vd_7      vd_8  ...   vd_1015   vd_1016   vd_1017  \
256  0.137267  0.024046  0.145395  ...  0.008003  0.013876  0.005360   
259  0.125938  0.033254  0.227433  ...  0.042140  0.036560  0.006585   
267  0.137267  0.024046  0.145395  ...  0.008003  0.013876  0.005360   
270  0.125938  0.033254  0.227433  ...  0.042140  0.036560  0.006585   
319  0.223354  0.045017  0.056177  ...  0.004973  0.000343  0.000000   

      vd_1018   vd_1019   vd_1020   vd_1021   vd_1022   vd_1023  y  
256  0.039292  0.029467  0.003972  0.0002

In [22]:

def data_split(df, pkl_list, test_size=0.3, validation_size=0.1, random_state=None):
    # Split into training and test sets
    train_set, test_set = train_test_split(pkl_list, test_size=test_size, random_state=random_state)

    # Further split the training set into training and validation sets
    train_set, validation_set = train_test_split(train_set, test_size=validation_size, random_state=random_state)

    train_idx = df[df['haim_id'].isin(train_set)]['haim_id'].tolist()
    validation_idx = df[df['haim_id'].isin(validation_set)]['haim_id'].tolist()
    test_idx = df[df['haim_id'].isin(test_set)]['haim_id'].tolist()

    x_train = df[df['haim_id'].isin(train_idx)].drop(['haim_id', 'y'], axis=1).values
    x_validation = df[df['haim_id'].isin(validation_idx)].drop(['haim_id', 'y'], axis=1).values
    x_test = df[df['haim_id'].isin(test_idx)].drop(['haim_id', 'y'], axis=1).values

    y_train = df[df['haim_id'].isin(train_idx)]['y'].values
    y_validation = df[df['haim_id'].isin(validation_idx)]['y'].values
    y_test = df[df['haim_id'].isin(test_idx)]['y'].values

    return x_train, x_validation, x_test, y_train, y_validation, y_test

In [23]:
x_train, x_val, x_test, y_train, y_val, y_test, = data_split(df, pkl_list)

In [10]:
# Prompt function to be fed into training loop

def formatting_func(example, emb, label):
    text = f"### INSTRUCTION: {'Use this input to create the correct label.'}\n### INPUT: {emb}\n### LABEL: {label}"
    return text

Creating Dataset and DataLoader

In [11]:
class CustomDataset(Dataset):
    def __init__(self, embedding, labels):
        self.labels = labels
        self.embedding = embedding

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        emb = self.embedding[idx]
        sample = {"Emb": emb, "Class": label}
        return sample

In [62]:
# collate_batch for strings

def collate_batch(batch):
     
    emb_list, classes = [], []
    for thing in batch:
        #print(batch)
        emb_list.append(thing['Emb'])
        classes.append(tokenizer(thing['Class'], return_tensors="pt"))
    text = torch.tensor(emb_list)
    classes = torch.cat([item['input_ids'] for item in classes], dim=0)
    return text, classes

In [12]:
# collate_batch for 0/1 labels

def collate_batch(batch):
     
    emb_list, classes = [], []
    for thing in batch:
        #print(batch)
        emb_list.append(thing['Emb'])
        classes.append(thing['Class'])
    text = torch.tensor(emb_list)
    classes = torch.tensor(classes, dtype=torch.float16)
    return text, classes

In [25]:
model = ProjectionNN()

Trainset = CustomDataset(x_train.tolist(), y_train)
Valset = CustomDataset(x_val, y_val)
Testset = CustomDataset(x_test, y_test)

Train_loader = DataLoader(Trainset, batch_size=1, collate_fn=collate_batch)
Val_loader = DataLoader(Valset, batch_size=1, collate_fn=collate_batch)
Test_loader = DataLoader(Testset, batch_size=1, collate_fn=collate_batch)

Training functions

In [14]:
def custom_output(emb, gemma):
    outputs = gemma(inputs_embeds=emb)
    noyes = [1294,3553]
    logits = outputs['logits']
    logits = logits[:,1,noyes]
    return logits

def output_to_label(logits):
    probs = torch.softmax(logits, dim=-1)
    predicted_token_id = torch.argmax(probs, dim=-1)
    return predicted_token_id

    
def train_epoch(model, gemma, optimizer, loss_fn, train_loader, device):
    # Train:
    model.train()
    train_loss_batches, train_acc_batches = [], []
    num_batches = len(train_loader)
    for batch_index, (x, y) in enumerate(train_loader, 1):
        inputs, labels = x.to(device), y.to(device)
        optimizer.zero_grad()

        emb = model(inputs)
        logits = custom_output(emb.to(torch.float16), gemma)
        
        loss = loss_fn(logits, labels.float())
        loss.backward()
        optimizer.step()
        train_loss_batches.append(loss.item())

        hard_preds = output_to_label(logits)
        acc_batch_avg = (hard_preds == labels).float().mean().item()
        train_acc_batches.append(acc_batch_avg)

    return model, train_loss_batches, train_acc_batches

def validate(model, gemma, loss_fn, val_loader, device, word_embs):
    val_loss_cum = 0
    val_acc_cum = 0
    model.eval()
    with torch.no_grad():
        for batch_index, (x, y) in enumerate(val_loader, 1):
            inputs, labels = x.to(device), y.to(device)
            emb = model.forward(inputs)
            concatted = torch.cat((word_embs,emb), dim=1).to(torch.float16)
            logits = custom_output(concatted, gemma)

            batch_loss = loss_fn(logits, labels.float())
            val_loss_cum += batch_loss.item()
            hard_preds = output_to_label(logits)
            acc_batch_avg = (hard_preds == labels).float().mean().item()
            val_acc_cum += acc_batch_avg
    return val_loss_cum/len(val_loader), val_acc_cum/len(val_loader)

In [15]:
def training_loop(model, gemma, optimizer, loss_fn, train_loader, val_loader, num_epochs):
    print("Starting training")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_losses, train_accs, val_losses, val_accs = [], [], [], []

    for epoch in range(1, num_epochs+1):
        model, train_loss, train_acc = train_epoch(model, gemma,
                                                   optimizer,
                                                   loss_fn,
                                                   train_loader,
                                                   device)
        val_loss, val_acc = validate(model, loss_fn, val_loader, device)
        print(f"Epoch {epoch}/{num_epochs}: "
              f"Train loss: {sum(train_loss)/len(train_loss):.3f}, "
              f"Train acc.: {sum(train_acc)/len(train_acc):.3f}, "
              f"Val. loss: {val_loss:.3f}, "
              f"Val. acc.: {val_acc:.3f}")
        train_losses.extend(train_loss)
        train_accs.extend(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
    return model, train_losses, train_accs, val_losses, val_accs

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()
training_loop(model, gemma, optimizer, loss_fn, Train_loader, Val_loader, 2)

Starting training


/home/edgelab/miniconda3/envs/gemma/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


ValueError: Target size (torch.Size([1])) must be the same as input size (torch.Size([1, 2]))

Experiments

In [ ]:
# Trying out finetune of generative aspect of gemma

import torch.nn.functional as F

embedding_matrix = gemma.get_input_embeddings().weight
prompt = "### PROMPT: Did this patient stay longer than 48 h?"
prompt_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
#print(prompt_ids["input_ids"].shape)
prompt_embeddings = [embedding_matrix[token_id].to('cuda') for token_id in prompt_ids["input_ids"][0]]
#print(lab.shape)
prompt_embeddings = torch.stack(prompt_embeddings)
label_embeddings = [embedding_matrix[token_id].to('cuda') for token_id in lab.to(dtype=torch.long)]
label_embeddings = torch.stack(label_embeddings)

p_mod = model(mod)

conc_emb = torch.cat([prompt_embeddings.unsqueeze(0).to(dtype=torch.float16), p_mod.to('cuda'), label_embeddings.to(dtype=torch.float16)], dim=1)



conc_emb.to(dtype=torch.float16).to('cuda')
output = gemma(inputs_embeds=conc_emb)


probabilities = F.softmax(output['logits'], dim=-1)
predicted_ids = torch.argmax(probabilities, dim=-1)

tokenizer.decode(predicted_ids[0])